<br>

# Municípios e Comarcas

Por meio do _site_ [ListaTelefonica](https://www.tjsp.jus.br/ListaTelefonica) foi possível relevar as APIs que operam e fazer diversas pesquisas.

<br>

Michel Metran\
Data: 18.06.2024\
Atualizado em: 18.06.2024


In [ ]:
import concurrent
import os

import pandas as pd
import requests
from dotenv import load_dotenv
from open_geodata import geo
from requests_ip_rotator import ApiGateway

from sp_tjsp_divadmin.my_paths import output_path_tab

In [ ]:
import pprint
import re
from typing import Literal

import numpy as np
from bs4 import BeautifulSoup
from more_itertools import one

from sp_tjsp_divadmin.my_functions import (
    find_text_between_parenthesis,
    keep_numbers,
    remove_accents,
    strip_accents,
)

<br>

---

## Lista de Municípios


Criamos função que pega o nome e Código do Município, de acordo com o TJSP.


In [ ]:
def get_lista_municipios_tjsp(municipio):
    """
    Pesquisa de municípios a partir de alguns caracteres.
    A função sempre retorna 10 itens.
    A cada caractere, o número de registros afunila!

    Exemplo de uso:
    df = get_lista_municipios_tjsp('Santos')

    :param municipio: _description_
    :type municipio: _type_
    :raises Exception: _description_
    """
    if len(municipio) < 3:
        raise Exception('A pesquisa de município deve ter mais de 3 caracteres')

    r = requests.post(
        'https://www.tjsp.jus.br/AutoComplete/ListarMunicipios',
        json={'texto': municipio},
    )
    if r.json() == 'listaVazia':
        pass

    else:
        df = pd.DataFrame(r.json())
        df = df.rename(
            mapper={
                'Codigo': 'id_municipio_tjsp',
                'Descricao': 'municipio_tjsp',
            },
            axis='columns',
        )
        return df

Para termos um input para a função, que é necessário inserir parte do nome, usamos os nomes dos municípios.


In [ ]:
# Cria Lista
df_geo_mun = geo.load_dataset('tab.sp.tab_municipio_nome')
lista_municipios = list(df_geo_mun['municipio_nome'])
lista_municipios[0:5]

In [ ]:
# Apenas para Testes
get_lista_municipios_tjsp(municipio='Santos')

<br>

Obtive o município com maior número de caracteres, visto que a função do método POST sempre lista apenas 10 municípios...


In [ ]:
n_caracteres_mun_max = max([len(x) for x in lista_municipios])
n_caracteres_mun_max

<br>

Uma vez com a função que retorna os municípios escrita, fiz uma iteração por todos os trechos de nomes de municípios. Para cada municípios foi pesquisado diversas vezes, incrementando o número de caracteres. Por exemplos, o município de Santos foi pesquisado:

- San
- Sant
- Santo
- Santos

<br>

Isso garantiu que todos os municípios foram "raspados".


In [ ]:
# list_dfs = []
list_termos = []
for i in range(n_caracteres_mun_max)[3:]:
    lista_municipios_temp = list(
        set([mun[:i] for mun in lista_municipios if len(mun) >= i])
    )
    for search_text in lista_municipios_temp:
        list_termos.append(search_text)


list_termos = list(set(list_termos))
print(f'São {len(list_termos)} termos para pesquisa')

Descobri uma maneira de paralelizar as consultas.\
Antes eu levava cerca de 40 minutos para rodar. Agora consigo fazer com 7 minutos.

É possível ainda deixar mais rápido, utilizando a API da AWS.


In [ ]:
MAX_THREADS = 5

with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_THREADS) as executor:
    temp = executor.map(get_lista_municipios_tjsp, list_termos)
    df_tjsp = pd.concat(list(temp), ignore_index=True)

# Resultados
df_tjsp.info()
df_tjsp.head()

Aqui com AWS


In [ ]:
if False:
    load_dotenv()

    aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
    aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')

    # Cria Gateway
    gateway = ApiGateway(
        site='https://www.tjsp.jus.br',
        access_key_id=aws_access_key_id,
        access_key_secret=aws_secret_access_key,
        regions=['sa-east-1'],
        verbose=True,
    )
    gateway.pool_connections = 10
    gateway.pool_maxsize = 10
    gateway.start()

    # Cria Session
    session = requests.Session()
    session.mount(prefix='https://www.tjsp.jus.br', adapter=gateway)

    # Parameters
    MAX_THREADS = 4
    list_dfs = []
    list_futures = []

    # Paralelo
    with concurrent.futures.ThreadPoolExecutor(
        max_workers=MAX_THREADS
    ) as executor:
        # Cria Lista de Tarefas
        for term in list_termos:
            futures = executor.submit(get_lista_municipios_tjsp, term)
            list_futures.append(futures)

        # Executa Lista de Tarefas
        for future in concurrent.futures.as_completed(list_futures):
            print(future.result())
            list_dfs.append(future.result())

    # Encerra o worker
    gateway.shutdown()

    # Crio a tabela
    df_tjsp = pd.concat(list_dfs, ignore_index=True)

Ajusto a tabela criada


In [ ]:
# Ajusta a tabela
df_tjsp = df_tjsp.drop_duplicates()
df_tjsp = df_tjsp.sort_values(by='municipio_tjsp')
df_tjsp = df_tjsp.iloc[
    df_tjsp['municipio_tjsp'].str.normalize('NFKD').argsort()
]
df_tjsp = df_tjsp.reset_index(drop=True)

# Resultados
df_tjsp.info()
df_tjsp.head()

In [ ]:
if len(df_tjsp) != 645:
    raise Exception('Falta Município!')

In [ ]:
filename = 'Municipios'

# Salva
df_tjsp.to_csv(output_path_tab / f'{filename}.csv', index=False)
df_tjsp.to_excel(
    output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
)

<br>

---

## Análise dos Nomes dos Municípios

A ideia principal é agregar os códigos do IBGE na tabela. Para isso eu fiz um join... vi os erros... e corrigi!


In [ ]:
# Read Data
df_tjsp = pd.read_csv(output_path_tab / 'Municipios.csv')

# Results
df_tjsp.info()
df_tjsp.head()

<br>

Peguei a tabela padrão, que fiz!, no projeto do OpenGeo!


In [ ]:
df_geo_mun = geo.load_dataset('tab.sp.tab_municipio_nome')
df_geo_mun.head()
df_geo_mun.info()

<br>

Juntei


In [ ]:
# Merge
df_merged = pd.merge(
    left=df_geo_mun,
    right=df_tjsp,
    left_on='municipio_nome',
    right_on='municipio_tjsp',
    how='left',
)

# Encontre erros
df_merged[df_merged['municipio_tjsp'].isnull()]

In [ ]:
# Fiz apenas consultas, para ver como está escrito no TJSP
df_tjsp[df_tjsp['municipio_tjsp'].str.startswith('Estrela')]
df_tjsp[df_tjsp['municipio_tjsp'].str.startswith('Luís')]
df_tjsp[df_tjsp['municipio_tjsp'].str.startswith('Florí')]

<br>

Uma vez que sei os erros dos nomes e são apenas 3 registros...
Criei uma coluna duplicada para corrigir!


In [ ]:
df_tjsp['municipio_corrigido'] = df_tjsp['municipio_tjsp']

# Ajuste os Valores Errados
df_tjsp['municipio_corrigido'] = df_tjsp['municipio_corrigido'].replace(
    {
        'Estrela dOeste': "Estrela d'Oeste",
        'Luís Antônio': 'Luiz Antônio',
        'Florínia': 'Florínea',
    }
)

In [ ]:
# Checa se foi substituido na coluna "Nome Corrigido"
df_tjsp[df_tjsp['municipio_corrigido'].str.startswith('Florí')]

<br>

Por fim, checo novamente e vejo que não tem mais nada nulo! Excelente!


In [ ]:
df_merged = pd.merge(
    left=df_geo_mun,
    right=df_tjsp,
    left_on='municipio_nome',
    right_on='municipio_corrigido',
    how='left',
)

# Confere se há algum erro
df_temp = df_merged[df_merged['municipio_tjsp'].isnull()]
if len(df_temp):
    raise Exception('Deu ruim')

# Results
df_merged.info()
df_merged.head()

In [ ]:
# Merge
df_merged = df_merged.drop(
    labels='municipio_nome', axis='columns', inplace=False, errors='ignore'
)

# Results
df_merged.info()
df_merged.head()

In [ ]:
filename = 'Municipios'

# Salva
df_merged.to_csv(output_path_tab / f'{filename}.csv', index=False)
df_merged.to_excel(
    output_path_tab / f'{filename}.xlsx', sheet_name=f'{filename}', index=False
)